In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import codecs
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

Using TensorFlow backend.


In [2]:
import argparse

sys.argv = ['test_rnn']

parser = argparse.ArgumentParser(description='LSTM text generation.')
parser.add_argument('--data', dest='data', type=str, help='data file')
parser.add_argument('--prefix', dest='prefix', type=str, help='output file prefix')
parser.add_argument('--vec', dest='vec', type=str, help='w2v model file')
parser.add_argument('--maxlen', dest='maxlen', type=int, help='sequence length')
parser.add_argument('--step', dest='step', type=int, help='sequence generated with step')
parser.add_argument('--layers', dest='layers', type=int, help='number of layers')
parser.add_argument('--lsize', dest='lsize', type=int, help='size of each layer')
parser.add_argument('--gen_length', dest='gen_length', type=int, help='length of text generated after each batch')
parser.add_argument('--batch', dest='batch', type=int, help='batch size in chars as read from file')
parser.add_argument('--model_batch', dest='model_batch', type=int, help='batch size in model training')
parser.add_argument('--dropout', dest='dropout', type=float, help='dropout prob between LSTM layers')
parser.add_argument('--gpu_fraction', dest='gpu_fraction', type=float, help='GPU memory fraction to be used by TensorFlow')
parser.set_defaults(
    data='/media/data/wiki.processed', prefix='model.lstm',
    lsize=512, layers=2, step=25, maxlen=100, dropout=0.2, batch=100, model_batch=256,
    gen_length=200, gpu_fraction=0.9,
)
args = parser.parse_args()


In [3]:
def get_session(gpu_fraction=0.3):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''

    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session(args.gpu_fraction))

In [4]:
chars = []
char_indices = {}
indices_char = {}
unic_chars = set()
cities = []
maxlen = 0

with open('../data/nlp/US_Cities.txt') as f:
    for line in f:
        line = line.strip()
        cities.append(line)
        if len(line) > maxlen:
            maxlen = len(line)

        for letter in line:
            unic_chars.add(letter)


i = 0
for c in unic_chars:
    chars.append(c)
    char_indices[c] = i
    indices_char[i] = c
    i += 1
chars = set(chars)
print('total chars:', len(chars))


total chars: 60


In [5]:
maxlen = 7

In [6]:
print('Build model...')
model = Sequential()
model.add(LSTM(maxlen, return_sequences=False, input_shape=(maxlen, len(chars))))
model.add(Dropout(args.dropout))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
json_string = model.to_json()
open('%s.json' % args.prefix, 'w').write(json_string)

Build model...


1650

In [ ]:
for iteration in range(1, 60):  
    print()
    print('-' * 50)
    print('Iteration', iteration)
    
    city_num = 0
    X = np.zeros((args.batch, maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((args.batch, len(chars)), dtype=np.bool)
    
    for city in cities:
        x_chars = []
        y_chars = []
        city += ' ' * (maxlen-len(city))
        
        for i in range(len(city) - maxlen - 1):
            x_chars = list(city[i:i+maxlen])
            y_chars = list(city[i+1:i+maxlen+1])
                
            for t, c in enumerate(x_chars):
                X[city_num, t, char_indices[c]] = 1
            y[city_num, char_indices[y_chars[-1]]] = 1
            
            city_num += 1
            if city_num == args.batch:
                model.fit(X, y, batch_size=args.batch, nb_epoch=1)
                model.save_weights('%s.i%d.h5' % (args.prefix, iteration), overwrite=True)
                city_num = 0
                X = np.zeros((args.batch, maxlen, len(chars)), dtype=np.bool)
                y = np.zeros((args.batch, len(chars)), dtype=np.bool)

    


--------------------------------------------------
Iteration 1
Epoch 1/1
100/100 [==============================] - 0s - loss: 2.9713
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.2949
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.5806
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.2916
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.2252
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.3811
Epoch 1/1


/usr/local/lib/python3.5/dist-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


100/100 [==============================] - 0s - loss: 3.3145
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.4483
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.4499
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.1661
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.3409
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.3007
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.4107
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.3309
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.3702
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.2768
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.2318
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.3333
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.4340
Epoch 1/1
100/100 [==============================] - 0s - loss: 3.1421
Epoch 1/1
100/10